In [1]:
%pwd
%cd ../..

/Users/ryandevera/data-science/umn_environments/Deeplifting


In [3]:
import pandas as pd
import glob

In [5]:
files = glob.glob('./results/*')

In [6]:
df = pd.read_parquet(files)

In [7]:
df

,x1,x2,x3,x4,x5,f,algorithm,time,problem_name,hits,dimensions
0,9.999991e-01,1.414273e+00,1.732078e+00,1.999955e+00,-2.236102e+00,9.238083e-08,IPOPT,0.044210,qing_5d,1,NaN
1,-9.999999e-01,1.414213e+00,1.732051e+00,2.000000e+00,-2.236068e+00,1.692854e-13,Dual Annealing,0.476093,qing_5d,1,5.0
2,-1.000000e+00,1.414214e+00,1.732051e+00,2.000000e+00,2.236068e+00,4.386560e-26,Differential Evolution,0.626581,qing_5d,1,5.0
3,1.570996e+01,7.853699e+01,-3.455717e+02,-1.193845e+02,4.115442e+02,3.379652e-21,PyGRANSO,1.004581,qing_5d,1,NaN
4,-8.199423e-03,9.999785e-01,1.414267e+00,1.732083e+00,-1.999957e+00,7.113237e-08,SCIP,7.066944,qing_5d,1,NaN
5,-7.316937e+00,-7.609311e+00,-9.746603e+00,-9.102135e-10,-9.980754e+00,-7.920279e+00,IPOPT,7.638671,layeb4_5d,0,NaN
6,-9.424959e+00,-1.683521e-10,9.424841e+00,-3.439338e-09,-3.140319e+00,-3.163097e+01,Dual Annealing,0.716211,layeb4_5d,0,5.0
7,-2.989934e+00,7.623993e-07,-3.128426e+00,9.236257e-06,3.066881e+00,-3.155550e+01,Differential Evolution,0.195509,layeb4_5d,0,5.0
8,1.229967e+00,-3.141593e+00,6.283185e+00,-3.173550e+01,9.424778e+00,-2.963102e+01,PyGRANSO,3.179992,layeb4_5d,0,NaN
9,4.166666e-09,3.140231e+00,1.584622e-16,-1.000000e+01,4.166666e-09,-1.081347e+02,SCIP,1.065467,layeb4_5d,0,NaN


In [10]:
df.groupby(['problem_name', 'algorithm'])['time'].agg(['count', 'mean'])

count       mean
problem_name      algorithm                               
ackley_100d       Differential Evolution      1   0.326528
                  Dual Annealing              1  13.493213
                  IPOPT                       1   0.132673
                  PyGRANSO                    1  31.464954
                  SCIP                        1   1.960092
ackley_30d        Differential Evolution      1  12.162284
                  Dual Annealing              1   3.578858
                  IPOPT                       1   0.044296
                  PyGRANSO                    1   1.807516
                  SCIP                        1   1.635487
ackley_3d         Differential Evolution      1   0.346068
                  Dual Annealing              1   0.388584
                  IPOPT                       1   0.021475
                  PyGRANSO                    1   0.143413
                  SCIP                        1   1.736442
ackley_5d         Differential Evolution      1   0.706947
                  Dual Annealing              1   0.600724
                  IPOPT                       1   0.043207
                  PyGRANSO                    1   0.125046
                  SCIP                        1   1.235282
alpine1_5d        Differential Evolution      1   0.667277
                  Dual Annealing              1   0.479355
                  IPOPT                       1   6.984651
                  PyGRANSO                    1   0.459868
                  SCIP                        1   3.707974
chung_reynolds_5d Differential Evolution      1   0.569883
                  Dual Annealing              1   0.424898
                  IPOPT                       1   0.047877
                  PyGRANSO                    1   2.636075
                  SCIP                        1   0.072432
griewank_5d       Differential Evolution      1   0.924796
                  Dual Annealing              1   0.576029
                  IPOPT                       1   0.067981
                  PyGRANSO                    1   0.286956
                  SCIP                        1   0.071077
layeb4_5d         Differential Evolution      1   0.195509
                  Dual Annealing              1   0.716211
                  IPOPT                       1   7.638671
                  PyGRANSO                    1   3.179992
                  SCIP                        1   1.065467
qing_5d           Differential Evolution      1   0.626581
                  Dual Annealing              1   0.476093
                  IPOPT                       1   0.044210
                  PyGRANSO                    1   1.004581
                  SCIP                        1   7.066944
rastrigin_5d      Differential Evolution      1   0.237309
                  Dual Annealing              1   0.466170
                  IPOPT                       1   0.051713
                  PyGRANSO                    1   0.229824
                  SCIP                        1   0.070742

In [ ]:
import torch
import numpy as np
from deeplifting.problems import HIGH_DIMENSIONAL_PROBLEMS_BY_NAME

problem = HIGH_DIMENSIONAL_PROBLEMS_BY_NAME['ackley_2500d']

In [ ]:
x = torch.zeros(2500)
results = np.zeros((1, 1, 2500 + 1))
trial = 0

In [ ]:
%%timeit
f = problem['objective']
f(x, results=results, trial=trial, version='pytorch')

In [ ]:
%%timeit
multi_result = [x] * 512
f_list = []
for xr in multi_result:
    r = f(xr, results=results, trial=trial, version='pytorch')
    f_list.append(r)

# Figure out performance

In [ ]:
import time

from pygranso.private.getNvar import getNvarTorch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct
from deeplifting.optimization import deeplifting_nd_fn, deeplifting_predictions
from deeplifting.utils import HaltLog

# first party
from deeplifting.models import DeepliftingSkipMLP

In [ ]:
def ndschwefel(x, version='pytorch'):
    """
    Implemention of the n-dimensional levy function

    Args:
    x: A d-dimensional array or tensor
    version: A string, either 'numpy' or 'pytorch'

    Returns:
    result: Value of the Rastrigin function
    """
    x = x.flatten()
    d = len(x)
    if version == 'numpy':
        result = 418.9829 * d - np.sum(x * np.sin(np.sqrt(np.abs(x))))
    elif version == 'pytorch':
        result = 418.9829 * d - torch.sum(x * torch.sin(torch.sqrt(torch.abs(x))))
    else:
        raise ValueError(
            "Unknown version specified. Available options are 'numpy' and 'pytorch'."
        )

    return result

In [ ]:
%%timeit
bounds = [(-32, 32)] * 2500

In [ ]:
# Deeplifting model with skip connections
model = DeepliftingSkipMLP(
    input_size=512,
    hidden_sizes=(512, 512, 512),
    output_size=2500,
    bounds=bounds,
    skip_every_n=1,
    activation='sine',
    output_activation='leaky_relu',
    agg_function='sum',
    seed=0,
)

In [ ]:
%%time
outputs = model(None)

In [ ]:
%%time
ndschwefel(outputs[0, :], version='pytorch')

In [ ]:
import tqdm


def deeplifting_nd_fn(model, objective, method='particle'):
    """
    Combined funtion used for PyGranso
    """
    outputs = model(inputs=None)

    # Get x1 and x2 so we can add the bounds
    # outputs = torch.sigmoid(outputs)
    # x = outputs.mean(axis=0)
    # print(f'Output x {x}')
    x, f = deeplifting_predictions(outputs, objective, method=method)

    return f

In [ ]:
%%time
fn = lambda x: objective(x, version='pytorch')
deeplifting_nd_fn(model, fn)

In [ ]:
device = torch.device('cpu')
dimensions = 2500
trials = 1
method = 'particle'
objective = ndschwefel

model = model.to(device=device, dtype=torch.double)
nvar = getNvarTorch(model.parameters())

# Setup a pygransoStruct for the algorithm
# options
opts = pygransoStruct()

# Inital x0
x0 = (
    torch.nn.utils.parameters_to_vector(model.parameters())
    .detach()
    .reshape(nvar, 1)
    .to(device=device, dtype=torch.double)
)

opts.x0 = x0
opts.torch_device = device
opts.print_frequency = 1
opts.limited_mem_size = 5
opts.stat_l2_model = False
opts.double_precision = True
opts.opt_tol = 1e-5
opts.maxit = 2

# results

# Set up the function with the results
fn = lambda x: objective(x, version='pytorch')  # noqa

# # Combined function
comb_fn = lambda model: deeplifting_nd_fn(
    model,
    fn,
    method=method,
)  # noqa

# Run the main algorithm
start_time = time.time()
soln = pygranso(var_spec=model, combined_fn=comb_fn, user_opts=opts)
end_time = time.time()
total_time = end_time - start_time

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm


def deeplifting_nd_fn(model, objective, method='particle'):
    """
    Combined funtion used for PyGranso
    """
    outputs = model(inputs=None)

    # Get x1 and x2 so we can add the bounds
    # outputs = torch.sigmoid(outputs)
    f = deeplifting_predictions(outputs, objective, method=method)

    return f


def deeplifting_predictions(x, objective, method='particle'):
    """
    Convert scaled values to the objective function
    """
    if method == 'particle':
        # Iterate over the objective values
        objective_values = []
        for i in range(len(x)):
            f = objective(x[i, :])
            objective_values.append(f)

        objective_values = torch.stack(objective_values)
        f = torch.min(objective_values)

    return f


bounds = [(-500, 500)] * 2
device = torch.device('cpu')

# Initialize model
model = DeepliftingSkipMLP(
    input_size=512,
    hidden_sizes=(1024, 1024, 1024),
    output_size=2,
    bounds=bounds,
    skip_every_n=1,
    activation='sine',
    output_activation='leaky_relu',
    agg_function='sum',
    seed=0,
)
model = model.to(device=device, dtype=torch.double)
model.train()

objective = ndschwefel

# Define the optimizer
# optimizer = optim.LBFGS(model.parameters(), lr=1e-3, line_search_fn='strong_wolfe')
optimizer = optim.Adam(model.parameters(), lr=1e-5)

criterion = lambda m: deeplifting_nd_fn(m, objective)

# Training loop
for epoch in range(250):
    # def closure():
    #     # Zero out the gradients
    #     optimizer.zero_grad()

    #     # The loss is the sum of the compliance
    #     loss = criterion(model)

    #     # Go through the backward pass and create the gradients
    #     loss.backward()

    #     return loss

    # # Step through the optimzer to update the data with the gradients
    # optimizer.step(closure)
    optimizer.zero_grad()

    loss = criterion(model)
    loss.backward()

    optimizer.step()

    print(criterion(model))

In [ ]:
%%timeit
criterion(model)

In [ ]:
%%time
criterion(model)